In [2]:
import cv2
import time
import matplotlib.pyplot as plt
import numpy as np
xxx = [] #获取人脸的坐标
yyy = [] #获取视频的图像
cap = cv2.VideoCapture(0)  # 调用第一个摄像头
# 人脸特征分类器
face_cascade = cv2.CascadeClassifier('C:/ProgramData/Anaconda3/Lib/site-packages/cv2/data/haarcascade_frontalface_alt2.xml')
# 循环识别
while True:
    # width = 1920 #定义摄像头获取图像宽度
    # height = 1080  #定义摄像头获取图像长度

    # cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)  #设置宽度
    # cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)  #设置长度
    ret, frame = cap.read()
    grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(grey, 1.3, 5)
    if faces != ():  #避免空数据的出现，如果没有人脸会返回空元组
        yyy.append(grey)
        xxx.append(faces)
    # 人脸处理，画矩形图像
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 1)
    # 显示图像以及窗口关闭
    cv2.imshow('kk', frame)
    flag = cv2.waitKey(1)
    if flag == 27:#按下ESC键退出
            break
# 销毁窗口
cv2.destroyAllWindows()

<ipython-input-2-734e58fd5e34>:20: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if faces != ():  #避免空数据的出现，如果没有人脸会返回空元组


In [3]:
#将BGR转化成RGB
def BGR2RGB(img):
    img=img.copy()
    t=img[:,:,0].copy()
    img[:,:,0]=img[:,:,2].copy()
    img[:,:,2]=t
    return img

In [4]:
import cv2
import os
def crop(img,set,i,name):  #裁剪函数
    num = set[0]
    cropped = img[num[1]+1:num[1]+num[3]-1,num[0]+1:num[0]+num[2]-1]  # 坐标为[y:y+h, x:x+w] +1/-1是为了不想看到外面那个蓝色的框框233333
    # plt.imshow(BGR2RGB(cropped))
    # plt.show()
    cv2.imwrite(f"E:\PycharmProjects\pythonProject\\rlsb\{name}\{name}.{i}.jpg",cropped) #保存文件
    return cropped

In [5]:
name = input("please enter name")
os.makedirs(f"E:\PycharmProjects\pythonProject\\rlsb\{name}")
for i in range(len(xxx)):
    crop(yyy[i],xxx[i],i,name)

In [6]:
for i in range(5):
    for (x, y, w, h) in xxx[i]:
            cv2.rectangle(yyy[i], (x, y), (x + w, y + h), (255, 0, 0), 2)
            # plt.imshow(BGR2RGB(yyy[i]))
            # plt.show()

In [7]:
import cv2
import os
import numpy as np
from PIL import Image
 
recognizer = cv2.face.LBPHFaceRecognizer_create()
detector = cv2.CascadeClassifier('C:/ProgramData/Anaconda3/Lib/site-packages/cv2/data/haarcascade_frontalface_alt2.xml')
 
 
def get_images_and_labels(path):
    path1 = [os.path.join(path, f) for f in os.listdir(path)]   #获取数据文件夹下面的文件夹的路径
    names = [f for f in os.listdir(path)]                       #获取数据文件夹下面的文件夹的名字
    #print(path1)
    face_samples = []                                           #人脸数据
    ids = []                                                    #数据标签
    for i in range(len(path1)):
        image_paths = [os.path.join(path1[i], f) for f in os.listdir(path1[i])]
        #print(image_paths)
    
        for image_path in image_paths:
            image = Image.open(image_path).convert("L")
            image_np = np.array(image, "uint8")
            faces = detector.detectMultiScale(image_np)
            for (x, y, w, h) in faces:
                face_samples.append(image_np[y : y + h, x : x + w])
                ids.append(i)
    
    return face_samples, ids ,names
    
 
faces, Ids,names = get_images_and_labels(r"rlsb\\")
# print(faces,Ids,names)
recognizer.train(faces, np.array(Ids))
recognizer.save("trainner.yml")

In [ ]:
import cv2
 
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainner.yml')
faceCascade = cv2.CascadeClassifier('C:/ProgramData/Anaconda3/Lib/site-packages/cv2/data/haarcascade_frontalface_alt2.xml')
font = cv2.FONT_HERSHEY_SIMPLEX
 
idnum = 0
 
cam = cv2.VideoCapture(0)
minW = 0.1*cam.get(3)
minH = 0.1*cam.get(4)
 
while True:
    # width = 1920 #定义摄像头获取图像宽度
    # height = 1080  #定义摄像头获取图像长度

    # cam.set(cv2.CAP_PROP_FRAME_WIDTH, width)  #设置宽度
    # cam.set(cv2.CAP_PROP_FRAME_HEIGHT, height)  #设置长度
    ret, img = cam.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
 
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.2,
        minNeighbors=5,
        minSize=(int(minW), int(minH))
    )
 
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
        idnum, confidence = recognizer.predict(gray[y:y+h, x:x+w])
 
        if confidence >40:              #置信大于40
            idnum = names[idnum]
            confidence = "{0}%".format(round(confidence))
        else:
            idnum = "unknown"
            confidence = "{0}%".format(round(confidence))
 
        cv2.putText(img, str(idnum), (x+5, y-5), font, 1, (0, 0, 255), 1)
        cv2.putText(img, str(confidence), (x+5, y+h-5), font, 1, (255, 255, 0), 1)
 
    cv2.imshow('camera', img)
    flag = cv2.waitKey(1)
    if flag == 27:                         #按下ESC键退出
            break
 
cam.release()
cv2.destroyAllWindows()